In [ ]:
"""
Purpose: To visualize how the proofreading performs on the manually proofread cells

Pseucode: 
For the example neuron id
1) get the neuron decomposition
2) Run the error detection to get the errored faces
3) Get the errored synapses
4) Plot them mesh and errored synapses



"""

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path
import neuron_visualizations as nviz


import datajoint_utils as du
from importlib import reload

INFO - 2020-12-01 00:31:21,914 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-01 00:31:21,916 - settings - Setting database.user to celiib
INFO - 2020-12-01 00:31:21,917 - settings - Setting database.password to newceliipass
INFO - 2020-12-01 00:31:21,921 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-12-01 00:31:21,922 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-01 00:31:21,932 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-12-01 00:31:22,130 - settings - Setting enable_python_native_blobs to True


In [2]:
import time
import datajoint_utils as du
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

from datajoint_utils import *

import error_detection as ed

minnie,schema = du.configure_minnie_vm()

INFO - 2020-12-01 00:31:22,151 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-01 00:31:22,352 - settings - Setting enable_python_native_blobs to True


In [3]:
m65 = minnie
decimation_version = 0
decimation_ratio = 0.25
key_source = (m65.Decimation().proj(decimation_version='version')  &
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)
              & m65.ManualProofreadIDs() & (m65.BaylorSegmentCentroid() & "multiplicity=1").proj())
nucleus_table =(m65.NucleusID & 'nucleus_version=1')  # Aug 1 timestamp
proofread_nucs = nucleus_table & m65.ManualProofreadIDs()
single_soma_nucs = nucleus_table & (dj.U('segment_id').aggr(proofread_nucs, n_nucs='count(nucleus_id)') & 'n_nucs=1').proj()
final_nucs = key_source* single_soma_nucs
final_nucs

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),nucleus_id id of segmented nucleus,nucleus_version segmentation version,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),size (EM voxels) scaled by (4x4x40),svid supervoxel_id of centroid
864691135564655959,0,0.25,225498,1,149712,151648,17549,1315328,85369450370640387
864691135748568361,0,0.25,230236,1,162848,184560,19809,1385955,87203504753932217
864691136194042326,0,0.25,258307,1,174944,137648,21113,1587826,88815663879212610
864691135233108569,0,0.25,262773,1,168096,161952,21503,1314296,87904100020764569
864691135740225387,0,0.25,264870,1,164832,177856,21208,1831649,87413717834430502
864691136105498585,0,0.25,269247,1,177536,213824,19894,1571057,89177746601154881
864691134988386682,0,0.25,269380,1,168016,216288,21534,1688056,87911453071322695
864691136333776819,0,0.25,271518,1,168464,226192,21121,1666190,87912758674384086
864691135771629819,0,0.25,294545,1,182976,135728,21116,1462702,89941288908168255
864691135748575017,0,0.25,296726,1,180976,143888,22162,1827886,89660913577301286


In [4]:
eligible_table = minnie.Decomposition & final_nucs.proj()

In [5]:
seg_ids_to_test = eligible_table.fetch("segment_id")
seg_ids_to_test

array([864691135564655959, 864691135748568361, 864691136194042326,
       864691135233108569, 864691135740225387, 864691136105498585,
       864691134988386682, 864691136333776819, 864691135771629819,
       864691136105484249, 864691135212632448, 864691136296650011,
       864691136311740477, 864691135491233631, 864691135945413668,
       864691135393964789, 864691135113167769, 864691136105493209,
       864691135272164113, 864691136618403213, 864691135974454639,
       864691136003930314, 864691135510455760, 864691136537220258,
       864691135785277636, 864691136333787571, 864691136105491417,
       864691135645592260, 864691136056391384, 864691136099807093,
       864691135348272855, 864691135737374100, 864691135212690816,
       864691136550489250, 864691136194103510, 864691135589907979,
       864691135407289801, 864691136545546146, 864691135382947307,
       864691136008425132, 864691135925564174, 864691136333790899,
       864691135974454895, 864691136008573614, 864691136134446

# Defining the Table to Store the statistics

In [6]:
@schema
class ManualProofreadValidationStats(dj.Manual):
    definition="""
    -> minnie.ManualProofreadIDs
    ---
    new_segment_id: bigint unsigned

    true_positive_presyn: int
    false_positive_presyn: int
    true_negative_presyn: int
    false_negative_presyn: int

    precision_presyn=NULL: double
    recall_presyn=NULL: double
    f1_presyn=NULL: double

    true_positive_postsyn: int
    false_positive_postsyn: int
    true_negative_postsyn: int
    false_negative_postsyn: int

    precision_postsyn=NULL: double
    recall_postsyn=NULL: double
    f1_postsyn=NULL: double

    true_positive_total : int
    false_positive_total : int
    true_negative_total : int
    false_negative_total : int

    precision_total=NULL: double
    recall_total=NULL: double
    f1_total=NULL: double

    """

In [7]:
ManualProofreadValidationStats()

segment_id segment id for those to be decimated,new_segment_id,true_positive_presyn,false_positive_presyn,true_negative_presyn,false_negative_presyn,precision_presyn,recall_presyn,f1_presyn,true_positive_postsyn,false_positive_postsyn,true_negative_postsyn,false_negative_postsyn,precision_postsyn,recall_postsyn,f1_postsyn,true_positive_total,false_positive_total,true_negative_total,false_negative_total,precision_total,recall_total,f1_total


In [8]:
plot_flag = False
only_consider_dendritic = True

In [10]:
for segment_id in seg_ids_to_test:
    # Doing the Actual Function


    key = dict(segment_id=segment_id)
    print(key)

    segment_id = key["segment_id"]
    global_start = time.time()

    verbose = True

    print(f"\n\n\n\n----- Working on {segment_id}-------")
    whole_pass_time = time.time()

    neuron_obj = (minnie.Decomposition() & key).fetch1("decomposition")
    errored_faces,axon_faces,non_axon_faces = ed.error_faces_by_axons(neuron_obj,verbose=True,
                                                                  visualize_errors_at_end=False,
                                                                 return_axon_non_axon_faces=True)
    
    print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")

    # Visualize the Actual Neuron

    if plot_flag:
        print(neuron_obj.segment_id)
        nviz.plot_objects(main_mesh=neuron_obj.mesh,
                     mesh_alpha=1)

    # Visualize the Results

    print(f"n_errored_faces = {len(errored_faces)}")

    ed = reload(ed)


    if neuron_obj is None:
        neuron_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")

    current_mesh = neuron_obj.mesh

    error_submesh = current_mesh.submesh([errored_faces],append=True)
    valid_mesh = tu.subtract_mesh(current_mesh,error_submesh)

    if plot_flag:
        nviz.plot_objects(main_mesh=valid_mesh,
                     meshes=error_submesh,
                      meshes_colors="red",
                     mesh_alpha=1)

    # How to get the percentage of synapses removed

    """
    Pseudocode: 
    1) Get the synapse file that corresponds to it
    2) Get the matching v7 id
    3) Get the v7 synapses (pre and post) that are still there
    4) Get the pre and post that are stil their after autoproofreading
    5) Things to find:
    - Eliminated and should have been (true positives)
    - Eliminated but SHOULDN'T (false positives) 
    - Not Eliminated and should have been not (true negative)
    - Not Eliminated but should have been eliminated (false negatie) 

    """

    # 1) Get the synapse file that corresponds to it
    data_path = Path("./manual_proofread_data/")
    current_file = data_path / Path(f"{segment_id}.pkl")
    current_file

    import system_utils as su
    curr_synapse_table = su.load_object(current_file)
    #eliminating the self connections
    
    # ---------- New Addition: Only Considering Dendritic Synapses ---------- #
    if only_consider_dendritic:
        
        
        
        print("Filtering out the axon synapses")
    
        df_pre_filt = curr_synapse_table[curr_synapse_table["pre_pt_root_id_Aug"] != curr_synapse_table["post_pt_root_id_Aug"]]

        from pykdtree.kdtree import KDTree
        # -------------- New Addittion: want to filter the synapses for only those on non-axonal faces ---- #

        #A) Get the synapse data for the august neuron 
        ctr_pt_position = df_pre_filt["ctr_pt_position"].to_numpy()
        synapse_ids = df_pre_filt["id"].to_numpy()
        synapse_centers = np.vstack(ctr_pt_position)
        synapse_centers_scaled = synapse_centers* [4, 4, 40]
        synapse_ids.shape,synapse_centers.shape

        #B) Filter to only those ids that are close to 


        current_mesh = neuron_obj.mesh
        neuron_kd = KDTree(current_mesh.triangles_center)
        dist,closest_face = neuron_kd.query(synapse_centers_scaled)

        neuron_mesh_labels = np.zeros(len(current_mesh.faces))
        neuron_mesh_labels[axon_faces] = 1
        closest_face_labels = neuron_mesh_labels[closest_face]

        not_considered_synapses_idx = np.where(closest_face_labels==1)[0]
        
        df = df_pre_filt[~df_pre_filt.id.isin(synapse_ids[not_considered_synapses_idx])]
        
    
    
    else:
        df = curr_synapse_table[curr_synapse_table["pre_pt_root_id_Aug"] != curr_synapse_table["post_pt_root_id_Aug"]]
    # ---------------------- End of Dendritic

    #1b) Get the synapse data for the august neuron 
    ctr_pt_position = df["ctr_pt_position"].to_numpy()
    synapse_ids = df["id"].to_numpy()
    synapse_centers = np.vstack(ctr_pt_position)
    synapse_ids.shape,synapse_centers.shape
    timestamps = np.zeros(len(synapse_ids))

    # 2) Get the matching v7 id
    import pandas as pd
    aug_to_v7_table = pd.read_csv("./proofread_cells_seg_correspondence_Aug1-v7.csv")
    v7_id = aug_to_v7_table[aug_to_v7_table["old_seg_id"]==segment_id]["new_seg_id"].tolist()[0]
    v7_id

    # check that the new v7 segment id is in the synapse table

    def get_pd_id_data(col_name,return_unique=False):
        ret_data = df [col_name].to_numpy()
        if return_unique:
            return np.unique(ret_data)
        else:
            return ret_data

    pre_pt_root_id = get_pd_id_data("pre_pt_root_id")
    pre_pt_root_id.shape,v7_id in pre_pt_root_id

    post_pt_root_id = get_pd_id_data("post_pt_root_id")
    post_pt_root_id.shape,v7_id in post_pt_root_id

    original_presyn = df[(df["pre_pt_root_id_Aug"] == segment_id)]["id"].to_numpy()
    original_postsyn = df[(df["post_pt_root_id_Aug"] == segment_id)]["id"].to_numpy()

    original_presyn.shape,original_postsyn.shape

    #3) Get the v7 synapses (pre and post) that are still there
    manual_remaining_presyn = df[(df["pre_pt_root_id"] == v7_id)]["id"].to_numpy()
    manual_remaining_postsyn = df[(df["post_pt_root_id"] == v7_id)]["id"].to_numpy()
    manual_remaining_presyn.shape,manual_remaining_postsyn.shape

    # find the non-remaining presyns and postsyns
    manual_error_presyn = np.setdiff1d(original_presyn,manual_remaining_presyn)
    manual_error_postsyn = np.setdiff1d(original_postsyn,manual_remaining_postsyn)

    manual_error_presyn.shape,manual_error_postsyn.shape

    len(manual_remaining_presyn) + len(manual_error_presyn), len(manual_remaining_postsyn) + len(manual_error_postsyn)

    #4) Get the pre and post that are stil their after autoproofreading
    ed = reload(ed)
    err_synapse_ids,non_err_synapse_ids = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                         errored_faces,return_synapse_ids=True,
                                                                       synapse_centers=synapse_centers,
                                                                       synapse_ids=synapse_ids
                                                                      )

    # divided the valid synapses into presyn and postsyn
    auto_remaining_presyn = np.intersect1d(original_presyn,non_err_synapse_ids)
    auto_remaining_postsyn = np.intersect1d(original_postsyn,non_err_synapse_ids)

    #dividing the error synapses into presyn and postsyn
    auto_error_presyn = np.intersect1d(original_presyn,err_synapse_ids)
    auto_error_postsyn = np.intersect1d(original_postsyn,err_synapse_ids)

    len(auto_remaining_presyn), len(auto_error_presyn), len(auto_remaining_postsyn), len(auto_error_postsyn)

    # Calculating the final scores

    # doing the presyn analysis
    true_positive_presyn = np.intersect1d(manual_error_presyn,auto_error_presyn) #deemed an error and was an error
    false_positive_presyn = np.intersect1d(manual_remaining_presyn,auto_error_presyn)#deemed an error but not an error

    true_negative_presyn = np.intersect1d(manual_remaining_presyn,auto_remaining_presyn) #not deemed an error and should not be an error
    false_negative_presyn = np.intersect1d(manual_error_presyn,auto_remaining_presyn) #not deemed and error but should be an error

    len(true_positive_presyn),len(false_positive_presyn),len(true_negative_presyn),len(false_negative_presyn)

    np.sum([len(true_positive_presyn),len(false_positive_presyn),len(true_negative_presyn),len(false_negative_presyn)])

    # Doing the postsyn analysis
    true_positive_postsyn = np.intersect1d(manual_error_postsyn,auto_error_postsyn) #deemed an error and was an error
    false_positive_postsyn = np.intersect1d(manual_remaining_postsyn,auto_error_postsyn)#deemed an error but not an error

    true_negative_postsyn = np.intersect1d(manual_remaining_postsyn,auto_remaining_postsyn) #not deemed an error and should not be an error
    false_negative_postsyn= np.intersect1d(manual_error_postsyn,auto_remaining_postsyn) #not deemed and error but should be an error

    len(true_positive_postsyn),len(false_positive_postsyn),len(true_negative_postsyn),len(false_negative_postsyn)

    np.sum([len(true_positive_postsyn),len(false_positive_postsyn),len(true_negative_postsyn),len(false_negative_postsyn)])

    #
    true_positive_total = list(true_positive_presyn) + list(true_positive_postsyn) 
    false_positive_total = list(false_positive_presyn) + list(false_positive_postsyn)

    true_negative_total = list(true_negative_presyn) + list(true_negative_postsyn)
    false_negative_total = list(false_negative_presyn) + list(false_negative_postsyn)

    len(true_positive_total),len(false_positive_total),len(true_negative_total),len(false_negative_total)

    np.sum([len(true_positive_total),len(false_positive_total),len(true_negative_total),len(false_negative_total)])

    def precision(TP,FP):
        if (TP + FP) > 0:
            return TP/(TP + FP)
        else:
            return np.nan

    def recall(TP,FN):
        if (TP + FN) > 0:
            return TP/(TP + FN)
        else:
            return np.nan

    def f1(TP,FP,FN):
        curr_prec = precision(TP,FP)
        curr_recall = recall(TP,FN)

        if curr_prec + curr_recall > 0:
            return 2*(curr_prec*curr_recall)/(curr_prec + curr_recall)
        else:
            return np.nan

    def calculate_scores(TP,FP,FN):
        return dict(precision=precision(TP,FP),
                   recall=recall(TP,FN),
                   f1=f1(TP,FP,FN))

    # calculating the accuracy, precision and f1

    #precision = (true positives)/(true positives + false positives)
    presyn_scores =  calculate_scores(len(true_positive_presyn),len(false_positive_presyn),len(false_negative_presyn))
    presyn_scores

    postsyn_scores =  calculate_scores(len(true_positive_postsyn),len(false_positive_postsyn),len(false_negative_postsyn))
    postsyn_scores

    total_scores =  calculate_scores(len(true_positive_total),len(false_positive_total),len(false_negative_total))
    total_scores

    # Visualize the Results


    err_synapses,non_err_synapses = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                         errored_faces,return_synapse_centroids=True,
                                                                       synapse_centers=synapse_centers,
                                                                       synapse_ids=synapse_ids
                                                                      )

    print(f"Segment ID = {segment_id}")
    print(f"n_synapses = {len(err_synapses) + len(non_err_synapses)}, n_errored_synapse = {len(err_synapses)}")


    valid_synapse_color = "yellow"
    error_color = "red"
    scatters_list = []
    scatters_colors = []

    if len(err_synapses) > 0:
        scatters_list.append(err_synapses)
        scatters_colors.append(error_color)
    if len(non_err_synapses) > 0:
        scatters_list.append(non_err_synapses)
        scatters_colors.append(valid_synapse_color)

    if plot_flag:
        if len(scatters_list) > 0:
            nviz.plot_objects(main_mesh=valid_mesh,
                      meshes=[error_submesh],
                              mesh_alpha=1,
                      meshes_colors=[error_color],
                    scatters=[err_synapses,non_err_synapses],
                     scatters_colors=[error_color,valid_synapse_color],
                             scatter_size=0.2)

        else:
            nviz.plot_objects(main_mesh=valid_mesh,
                     meshes=error_submesh,
                      meshes_colors=error_color,
                     mesh_alpha=1)


    # Write the Results to a Datajoint Table


    info_key = dict(segment_id=segment_id,
                   new_segment_id=v7_id,

                    true_positive_presyn = len(true_positive_presyn),
                    false_positive_presyn = len(false_positive_presyn),
                    true_negative_presyn= len(true_negative_presyn),
                    false_negative_presyn = len(false_negative_presyn),

                    precision_presyn = presyn_scores["precision"],
                    recall_presyn = presyn_scores["recall"],
                    f1_presyn = presyn_scores["f1"],

                    true_positive_postsyn = len(true_positive_postsyn),
                    false_positive_postsyn = len(false_positive_postsyn),
                    true_negative_postsyn = len(true_negative_postsyn),
                    false_negative_postsyn = len(false_negative_postsyn),

                    precision_postsyn = postsyn_scores["precision"],
                    recall_postsyn = postsyn_scores["recall"],
                    f1_postsyn = postsyn_scores["f1"],

                    true_positive_total= len(true_positive_total),
                    false_positive_total = len(false_positive_total),
                    true_negative_total = len(true_negative_total),
                    false_negative_total = len(false_negative_total),

                    precision_total = total_scores["precision"],
                    recall_total=total_scores["recall"],
                    f1_total = total_scores["f1"],


                   )
    print(f"total_scores = {total_scores}")

    ManualProofreadValidationStats.insert1(info_key,skip_duplicates=True)

{'segment_id': 864691135564655959}




----- Working on 864691135564655959-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f12759337b8>, <function median_mesh_center at 0x7f1275933a60>, <function n_spines at 0x7f12759338c8>, <function n_faces_branch at 0x7f1277a6a488>, <function skeleton_distance_branch at 0x7f1275933840>, <function spines_per_skeletal_length at 0x7f1275933d08>, <function no_spine_median_mesh_center at 0x7f1275933b70>]
functions_list = [<function axon_segment at 0x7f1275934400>]
Axons not keeping because of soma: {'L0': array([17, 18, 20, 21])}

----- Working on L1 ------


For limb 1 the valid axon branches are []
The following are not valid: []

----- Working on L4 ------
-- Axon Group 0 of size 35--
   Working on soma 0, starting_node 34
Not using AIS angle threshold
sk_angle= 16.380534881177304


For limb 4 the valid axon branches are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 2

Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f12759337b8>, <function median_mesh_center at 0x7f1275933a60>, <function n_spines at 0x7f12759338c8>, <function n_faces_branch at 0x7f1277a6a488>, <function skeleton_distance_branch at 0x7f1275933840>, <function spines_per_skeletal_length at 0x7f1275933d08>, <function no_spine_median_mesh_center at 0x7f1275933b70>]
functions_list = [<function axon_segment at 0x7f1275934400>]
Axons not keeping because of soma: {'L0': array([], dtype=int64)}

----- Working on L1 ------


For limb 1 the valid axon branches are []
The following are not valid: []

----- Working on L2 ------


For limb 2 the valid axon branches are []
The following are not valid: []

----- Working on L3 ------
-- Axon Group 0 of size 31--
   Working on soma 0, starting_node 6
Not using AIS angle threshold
sk_angle= 13.39553702962113


For limb 3 the valid axon branches are [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 1



 -------- Total number of error faces = 80527 --------------
Computing the axon and non-axonal faces


 ------ Total time for 864691136105498585 = 46.62267017364502 ------
n_errored_faces = 80527
Filtering out the axon synapses
Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691136105498585
n_synapses = 824, n_errored_synapse = 208
total_scores = {'precision': 0.0, 'recall': 0.0, 'f1': nan}
{'segment_id': 864691134988386682}




----- Working on 864691134988386682-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f12759337b8>, <function median_mesh_center at 0x7f1275933a60>, <function n_spines at 0x7f12759338c8>, <function n_faces_branch at 0x7f1277a6a488>, <function skeleton_distance_branch at 0x7f1275933840>, <function spines_per_skeletal_length at 0x7f1275933d08>, <function no_spine_median_mesh_center

Axons not keeping because of soma: {'L0': array([18, 20]), 'L4': array([3])}

----- Working on L1 ------
-- Axon Group 0 of size 70--
   Working on soma 0, starting_node 74
Not using AIS angle threshold
sk_angle= 19.696482264518938


For limb 1 the valid axon branches are [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 56 57 58 59 60 61 62 64 65 66 67 68 69 70 71 73 80]
The following are not valid: []

----- Working on L2 ------
-- Axon Group 0 of size 25--
   Working on soma 0, starting_node 2
Not using AIS angle threshold
sk_angle= 108.43143268294908
*****Path to axon group not valid******
-- Axon Group 1 of size 9--
   Working on soma 0, starting_node 2
Not using AIS angle threshold
sk_angle= 108.43143268294908
*****Path to axon group not valid******
-- Axon Group 2 of size 3--
   Working on soma 0, starting_node 2
Not using AIS angle threshold
sk_angle= 108.431432682949

Not using AIS angle threshold
sk_angle= 12.11673145024742
-- Axon Group 3 of size 102--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 12.11673145024742


For limb 0 the valid axon branches are [129 162 110  79  80  87  88  89 134 139 143 145 152 153 158  30  32  33
  34  31 164  45  46  47  49  50  51  60  73  74  81  82  90  91 115 116
 117 118 119 120 124 125   0   1 131 132 101 133 103 142 111  92 113  93
  25  26  27  28  29  94  95   5   6   7   8   9  10  11  12  13  14  15
  16  17  18  19  20  21  22  23  24  35  36  37  38  39  40  41  42  43
  44  48  52  53  54  55  56  57  58  59  61  62  63  64  65  66  67  68
  69  70  71  72  75  76  77  78  83  84  85  86  96  97  98  99 100 102
 104 105 106 107 108 109 112 114 121 122 123 126 127 128 130 135 136 137
 138 140 141 144 146 147 148 149 150 151 154 155 156 157 159 160 161 163
 165]
The following are not valid: []

----- Working on L2 ------
-- Axon Group 0 of size 10--
   Working on soma 0, s

...

Not using AIS angle threshold
sk_angle= 83.37232546480284
*****Path to axon group not valid******
-- Axon Group 1 of size 6--
   Working on soma 0, starting_node 0
Not using AIS angle threshold
sk_angle= 83.37232546480284
*****Path to axon group not valid******


For limb 6 the valid axon branches are []
The following are not valid: [5, 7, 8, 9, 10, 11, 12]

----- Working on L7 ------


For limb 7 the valid axon branches are []
The following are not valid: []
final_error_axons = {'L0': array([53, 54, 55, 56, 57, 59, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]), 'L5': array([ 3,  4,  5,  6,  7,  8,  9, 11, 12]), 'L8': array([], dtype=int64), 'L1': [], 'L10': [], 'L12': [], 'L2': [], 'L4': [], 'L6': [5, 7, 8, 9, 10, 11, 12], 'L7': []}


 -------- Total number of error faces = 142669 --------------
Computing the axon and non-axonal faces


 ------ Total time for 864691136134446219 = 125.60976052284241 ------
n_errored_faces = 142669
Filtering out the axon synapses
Returning the synpase id



 -------- Total number of error faces = 984 --------------
Computing the axon and non-axonal faces


 ------ Total time for 864691136537477538 = 55.150951623916626 ------
n_errored_faces = 984
Filtering out the axon synapses
Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691136537477538
n_synapses = 6665, n_errored_synapse = 3
total_scores = {'precision': 0.6666666666666666, 'recall': 0.11764705882352941, 'f1': 0.2}
{'segment_id': 864691135865518469}




----- Working on 864691135865518469-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f12759337b8>, <function median_mesh_center at 0x7f1275933a60>, <function n_spines at 0x7f12759338c8>, <function n_faces_branch at 0x7f1277a6a488>, <function skeleton_distance_branch at 0x7f1275933840>, <function spines_per_skeletal_length at 0x7f1275933d08>, <function 



 -------- Total number of error faces = 706 --------------
Computing the axon and non-axonal faces


 ------ Total time for 864691136311791677 = 95.73164677619934 ------
n_errored_faces = 706
Filtering out the axon synapses
Returning the synpase ids 1) errored synpases 2) non-errored synapses
Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691136311791677
n_synapses = 6925, n_errored_synapse = 3
total_scores = {'precision': 0.0, 'recall': 0.0, 'f1': nan}
{'segment_id': 864691135735426708}




----- Working on 864691135735426708-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f12759337b8>, <function median_mesh_center at 0x7f1275933a60>, <function n_spines at 0x7f12759338c8>, <function n_faces_branch at 0x7f1277a6a488>, <function skeleton_distance_branch at 0x7f1275933840>, <function spines_per_skeletal_length at 0x7f1275933d08>, <function no_spine_median_mesh_center at 0

Returning the 1) coordinates for errored synapses 2) Coordinates for non-errored synapses
Segment ID = 864691136370815112
n_synapses = 3977, n_errored_synapse = 160
total_scores = {'precision': 0.0, 'recall': nan, 'f1': nan}
{'segment_id': 864691135699269154}




----- Working on 864691135699269154-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f12759337b8>, <function median_mesh_center at 0x7f1275933a60>, <function n_spines at 0x7f12759338c8>, <function n_faces_branch at 0x7f1277a6a488>, <function skeleton_distance_branch at 0x7f1275933840>, <function spines_per_skeletal_length at 0x7f1275933d08>, <function no_spine_median_mesh_center at 0x7f1275933b70>]
functions_list = [<function axon_segment at 0x7f1275934400>]
Axons not keeping because of soma: {'L1': array([16, 17, 18, 19, 20, 25, 26, 27, 28, 29, 31, 32, 35, 38, 39, 40]), 'L5': array([], dtype=int64)}

----- Working on L0 ------
-- Axon Group 0 of size 5--
   Working on so

Not using AIS angle threshold
sk_angle= 136.64719266991108
*****Path to axon group not valid******
   Working on soma 0, starting_node 23
Using AIS angle threshold 110
sk_angle= 101.20351400428075
-- Axon Group 2 of size 1--
   Working on soma 0, starting_node 10
Not using AIS angle threshold
sk_angle= 136.64719266991108
*****Path to axon group not valid******
   Working on soma 0, starting_node 23
Not using AIS angle threshold
sk_angle= 116.66652665381594
*****Path to axon group not valid******
   Working on soma 0, starting_node 39
Not using AIS angle threshold
sk_angle= 106.6193100347111
*****Path to axon group not valid******
   Working on soma 0, starting_node 47
Not using AIS angle threshold
sk_angle= 88.70203725939565
*****Path to axon group not valid******
   Working on soma 0, starting_node 10
Not using AIS angle threshold
sk_angle= 136.64719266991108
*****Path to axon group not valid******
   Working on soma 0, starting_node 23
Not using AIS angle threshold
sk_angle= 116.6665

In [ ]:
ManualProofreadValidationStats()